##Solutions to Practice Example Chapter 6##

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pysal

In [3]:
db = pysal.open('data/natregimes.dbf','r')

In [4]:
len(db)

3085

In [5]:
db.header

['REGIONS',
 'NOSOUTH',
 'POLY_ID',
 'NAME',
 'STATE_NAME',
 'STATE_FIPS',
 'CNTY_FIPS',
 'FIPS',
 'STFIPS',
 'COFIPS',
 'FIPSNO',
 'SOUTH',
 'HR60',
 'HR70',
 'HR80',
 'HR90',
 'HC60',
 'HC70',
 'HC80',
 'HC90',
 'PO60',
 'PO70',
 'PO80',
 'PO90',
 'RD60',
 'RD70',
 'RD80',
 'RD90',
 'PS60',
 'PS70',
 'PS80',
 'PS90',
 'UE60',
 'UE70',
 'UE80',
 'UE90',
 'DV60',
 'DV70',
 'DV80',
 'DV90',
 'MA60',
 'MA70',
 'MA80',
 'MA90',
 'POL60',
 'POL70',
 'POL80',
 'POL90',
 'DNL60',
 'DNL70',
 'DNL80',
 'DNL90',
 'MFIL59',
 'MFIL69',
 'MFIL79',
 'MFIL89',
 'FP59',
 'FP69',
 'FP79',
 'FP89',
 'BLK60',
 'BLK70',
 'BLK80',
 'BLK90',
 'GI59',
 'GI69',
 'GI79',
 'GI89',
 'FH60',
 'FH70',
 'FH80',
 'FH90',
 'West']

In [6]:
y_name = 'HR60'
y = np.array([db.by_col(y_name)]).T

In [7]:
x_names = ['RD60','MA60','PS60']
x = np.array([db.by_col(var) for var in x_names]).T

In [8]:
yend_names = ['UE60']
yend = np.array([db.by_col(var) for var in yend_names]).T

In [9]:
q_names = ['FH60','FP59','GI59']
q = np.array([db.by_col(var) for var in q_names]).T

In [10]:
w = pysal.knnW_from_shapefile('data/natregimes.shp',
                                k=6,radius=pysal.cg.RADIUS_EARTH_MILES,idVariable='FIPSNO')

In [11]:
w.n

3085

In [12]:
w.transform = 'r'

In [13]:
kw = pysal.adaptive_kernelW_from_shapefile('data/natregimes.shp',
                                             k=12,function='quadratic',radius=pysal.cg.RADIUS_EARTH_MILES,
                                             diagonal=True,idVariable='FIPSNO')

In [14]:
kw.n

3085

two stage least squares with White standard errors and diagnostics for spatial dependence

In [15]:
reg1 = pysal.spreg.TSLS(y,x,yend,q,w=w,robust='white',spat_diag=True,
                       name_y=y_name,name_x=x_names,name_yend=yend_names,
                       name_q=q_names,name_w='nat_k6',
                       name_ds='natregimes.shp')

In [16]:
print reg1.summary

REGRESSION
----------
SUMMARY OF OUTPUT: TWO STAGE LEAST SQUARES
------------------------------------------
Data set            :natregimes.shp
Weights matrix      :      nat_k6
Dependent Variable  :        HR60                Number of Observations:        3085
Mean dependent var  :      4.5041                Number of Variables   :           5
S.D. dependent var  :      5.6497                Degrees of Freedom    :        3080
Pseudo R-squared    :      0.2044

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      13.4510208       0.9883561      13.6094882       0.0000000
                MA60      -0.2485547       0.0212910     -11.6741458       0.0000000
                PS60       0.4393596       0.1330675       3.3017812       0.0009607


two stage least squares with HAC standard errors and diagnostics for spatial dependence

In [17]:
reg2 = pysal.spreg.TSLS(y,x,yend,q,w=w,robust='hac',gwk=kw,spat_diag=True,
                       name_y=y_name,name_x=x_names,name_yend=yend_names,
                       name_q=q_names,name_w='nat_k6',name_gwk='nat_quadratic_12',
                       name_ds='natregimes.shp')

In [18]:
print reg2.summary

REGRESSION
----------
SUMMARY OF OUTPUT: TWO STAGE LEAST SQUARES
------------------------------------------
Data set            :natregimes.shp
Weights matrix      :      nat_k6
Dependent Variable  :        HR60                Number of Observations:        3085
Mean dependent var  :      4.5041                Number of Variables   :           5
S.D. dependent var  :      5.6497                Degrees of Freedom    :        3080
Pseudo R-squared    :      0.2044

HAC Standard Errors; Kernel Weights: nat_quadratic_12
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      13.4510208       1.1336621      11.8651063       0.0000000
                MA60      -0.2485547       0.0244053     -10.1844645       0.0000000
                PS60       0.4393596       0.1461997 